<a href="https://colab.research.google.com/github/zhaizeyu/test/blob/master/train_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p '/root/.kaggle'
from google.colab import userdata

# 打开文件以写入，如果文件不存在则创建
with open('/root/.kaggle/kaggle.json','w') as file:
    f = {"username":userdata.get('KAGGLE_USERNAME'),"key":userdata.get('KAGGLE_KEY')}
    # 写入内容 'ccc' 到文件
    file.write('{"username":"zeyuzhai","key":"42dc12af61163cb7e7c0a849dd7e8f49"}')
!chmod 600 '/root/.kaggle/kaggle.json'